# Notebook error analysis 

## Refining error types - Word-embeddings

retrain word embeddings

### Prepare dataset for re-training word embeddings
Construct a dataset for retraining word embeddings. It needs all possible data (as much data as possible), no filtering, keep the preprocess part the same. Using traceback because it may cover more words..

In [1]:
import pandas as pd
import json
import config, util

path_err_k = config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx')
df_err_k = pd.read_excel(path_err_k)
path_err_g = config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx')
df_err_g = pd.read_excel(path_err_g)

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [2]:
df_err_evalue = pd.concat([df_err_k[["fname","eid","ename","ename_mapped","evalue"]], df_err_g[["fname","eid","ename","ename_mapped","evalue"]]])
df_err_traceback = pd.concat([df_err_k[["fname","eid","ename","ename_mapped","traceback"]], df_err_g[["fname","eid","ename","ename_mapped","traceback"]]])

In [5]:
import cluster_util

df_err_traceback["traceback_tokenized"] = df_err_traceback.traceback.apply(cluster_util.preprocess_text)
df_err_evalue["evalue_tokenized"] = df_err_evalue.evalue.apply(cluster_util.preprocess_text)

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df_err_evalue.to_excel(config.path_default.joinpath('nberror_evalue.xlsx'), index=False, engine='xlsxwriter')
df_err_traceback.to_excel(config.path_default.joinpath('nberror_traceback.xlsx'), index=False, engine='xlsxwriter')

### Re-training word embeddings with Word2Vec and FastText

In [1]:
import pandas as pd
import json
import config, util
import retrain_word2vec

df_err_evalue = pd.read_excel(config.path_default.joinpath('nberror_evalue.xlsx'))
df_err_traceback = pd.read_excel(config.path_default.joinpath('nberror_traceback.xlsx'))

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [2]:
tokenized_traindata = list()
tokenized_traindata.extend(df_err_evalue.evalue_tokenized.to_list())
tokenized_traindata.extend(df_err_traceback.traceback_tokenized.to_list())

In [3]:
tokenized_traindata = [str(ele).split(" ") for ele in tokenized_traindata]

In [22]:
import importlib
importlib.reload(retrain_word2vec)
import retrain_word2vec

In [11]:
import cluster_util

# w2v_model = retrain_word2vec.retrain_word2vec(tokenized_traindata, config.path_w2v_models, vector_size=200)

# glove_vectors = cluster_util.load_glove(config.path_default.joinpath("glove.6B/glove.6B.200d.txt"))
# sw2v_model = retrain_word2vec.finetune_word2vec(tokenized_traindata, glove_vectors, config.path_w2v_models, 
#                                                         modelname="nberr_word2vec_glove_finetune.model",vector_size=200)

# sw2v_model = retrain_word2vec.retrain_subword2vec(tokenized_traindata, config.path_w2v_models, vector_size=200)

binpath = r"C:\Users\yirwa29\Downloads\Dataset-Nb/wiki.en.bin"
sw2v_model = retrain_word2vec.finetune_subword2vec(tokenized_traindata, binpath, config.path_w2v_models, vector_size=200)

Training has finished. Model saved in file.


In [12]:
print(len(sw2v_model.wv["tensor"]))
sw2v_model.wv.most_similar(["tensor"])

200


[('dtensor', 0.9035197496414185),
 ('qtensor', 0.8982171416282654),
 ('adptensor', 0.8850652575492859),
 ('imtensor', 0.8759932518005371),
 ('sttensor', 0.8726184368133545),
 ('ytensor', 0.8705863356590271),
 ('ckkstensor', 0.8650293350219727),
 ('tensorz', 0.8645163774490356),
 ('dltensor', 0.8611711263656616),
 ('xtensor', 0.8575398325920105)]

In [24]:
## usage
# import cluster_util,retrain_word2vec,config
# w2v_model = retrain_word2vec.load_word2vec(config.path_w2v_models)
# cluster_util.vectorizer_word2vec("tensor shape mismatch", w2v_model.wv, w2v_model.vector_size, aggregation='mean')